---
title: "Econometrics Code for Simulation"
theme:
  light: lux

jupyter: python3
---

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

def pension_fund_simulation():
    global df  # Make the DataFrame global so that it can be used in Cell 2

    def get_auto_enrolment_rate(year):
        """
        Returns the employee contribution rate (as a decimal)
        for a given 'scheme year' following the staged rates:
          - Years 1-3: 1.5%
          - Years 4-6: 3.0%
          - Years 7-9: 4.5%
          - Year 10 and after: 6.0%
        """
        if 1 <= year <= 3:
            return 0.015
        elif 4 <= year <= 6:
            return 0.03
        elif 7 <= year <= 9:
            return 0.045
        else:
            return 0.06

    # Get user inputs
    starting_salary = float(input("Enter starting salary (€): "))
    salary_increase_rate_early = float(input("Enter early salary increase rate (e.g., 0.10 for 10%): "))
    salary_increase_rate_late = float(input("Enter late salary increase rate after 5 years (e.g., 0.02 for 2%): "))
    pension_contribution_rate = float(input("Enter pension contribution rate for 'CUAN' scenario (e.g., 0.10 for 10%): "))
    increase_contribution_rate = float(input("Enter percentage of salary increase allocated to pension for first 5 years (e.g., 0.60 for 60%): "))
    investment_return = float(input("Enter annual investment return rate (e.g., 0.07 for 7%): "))
    fee_rate = float(input("Enter annual pension fund fee rate (e.g., 0.01 for 1%): "))
    years = int(input("Enter number of years to simulate: "))

    # Initialize lists (starting with Year 0)
    years_list = [0]
    salary_list = [starting_salary]

    # Scenario A: "CUAN" (increased contributions)
    annual_contribution_cuan = pension_contribution_rate * starting_salary
    pension_balance_cuan = annual_contribution_cuan
    fees_accumulated_cuan = 0

    # Scenario B: "Auto-Enrolment"
    auto_rate = get_auto_enrolment_rate(1)
    annual_contribution_auto = auto_rate * starting_salary
    pension_balance_auto = annual_contribution_auto
    fees_accumulated_auto = 0

    # Store initial values
    annual_contribution_cuan_list = [annual_contribution_cuan]
    annual_contribution_auto_list = [annual_contribution_auto]
    pension_balance_cuan_list = [pension_balance_cuan]
    pension_balance_auto_list = [pension_balance_auto]
    fees_accumulated_cuan_list = [fees_accumulated_cuan]
    fees_accumulated_auto_list = [fees_accumulated_auto]
    pension_after_fees_cuan_list = [pension_balance_cuan]  # At Year 0, before fees = after fees
    pension_after_fees_auto_list = [pension_balance_auto]

    current_salary = starting_salary
    for yr in range(1, years + 1):
        # Increase salary and update CUAN contributions
        if yr <= 5:
            salary_increase = current_salary * salary_increase_rate_early
            additional_contribution = salary_increase * increase_contribution_rate
            annual_contribution_cuan += additional_contribution
            current_salary *= (1 + salary_increase_rate_early)
        else:
            current_salary *= (1 + salary_increase_rate_late)
            annual_contribution_cuan = max(annual_contribution_cuan, current_salary * pension_contribution_rate)

        # Auto-Enrolment: update contribution using staged rate
        auto_rate = get_auto_enrolment_rate(yr)
        annual_contribution_auto = current_salary * auto_rate

        # CUAN scenario: update balance and fees
        pension_balance_cuan = (pension_balance_cuan + annual_contribution_cuan) * (1 + investment_return)
        fees_paid_cuan = pension_balance_cuan * fee_rate
        pension_balance_cuan_after_fees = pension_balance_cuan - fees_paid_cuan
        fees_accumulated_cuan += fees_paid_cuan

        # Auto-Enrolment scenario: update balance and fees
        pension_balance_auto = (pension_balance_auto + annual_contribution_auto) * (1 + investment_return)
        fees_paid_auto = pension_balance_auto * fee_rate
        pension_balance_auto_after_fees = pension_balance_auto - fees_paid_auto
        fees_accumulated_auto += fees_paid_auto

        # Append results for this year
        years_list.append(yr)
        salary_list.append(current_salary)
        annual_contribution_cuan_list.append(annual_contribution_cuan)
        annual_contribution_auto_list.append(annual_contribution_auto)
        pension_balance_cuan_list.append(pension_balance_cuan)
        pension_balance_auto_list.append(pension_balance_auto)
        fees_accumulated_cuan_list.append(fees_accumulated_cuan)
        fees_accumulated_auto_list.append(fees_accumulated_auto)
        pension_after_fees_cuan_list.append(pension_balance_cuan_after_fees)
        pension_after_fees_auto_list.append(pension_balance_auto_after_fees)

    # Create the DataFrame and keep the full table
    df = pd.DataFrame({
        "Year": years_list,
        "Salary (€)": salary_list,
        "Annual Contribution (€) (CUAN)": annual_contribution_cuan_list,
        "Annual Contribution (€) (Auto-Enrolment)": annual_contribution_auto_list,
        "Pension Balance Before Fees (€) (CUAN)": pension_balance_cuan_list,
        "Pension Balance Before Fees (€) (Auto-Enrolment)": pension_balance_auto_list,
        "Pension Balance After Fees (€) (CUAN)": pension_after_fees_cuan_list,
        "Pension Balance After Fees (€) (Auto-Enrolment)": pension_after_fees_auto_list,
        "Total Fees Earned (€) (CUAN)": fees_accumulated_cuan_list,
        "Total Fees Earned (€) (Auto-Enrolment)": fees_accumulated_auto_list
    })

    pd.options.display.float_format = '{:,.2f}'.format
    # Print the full table
    print("Full Simulation Table:")
    print(df.to_string(index=False))

    # Plot the results for interactivity
    fig = px.line(
        df,
        x="Year",
        y=[
            "Pension Balance After Fees (€) (CUAN)",
            "Pension Balance After Fees (€) (Auto-Enrolment)",
            "Total Fees Earned (€) (CUAN)",
            "Total Fees Earned (€) (Auto-Enrolment)"
        ],
        labels={"value": "Amount (€)", "Year": "Years"},
        title="Pension Balance Growth & Fees Comparison: CUAN vs Auto-Enrolment"
    )
    fig.update_layout(yaxis_tickformat=",")
    fig.show()

# Run the simulation cell
pension_fund_simulation()

In [ ]:
# Ensure we have our global DataFrame 'df'
if 'df' not in globals():
    print("Error: Simulation DataFrame 'df' not found. Please run the simulation cell first.")
else:
    # Use a copy of df for stats (this does not alter the original)
    df_stats = df.copy()

    # Compute stats based on the final row
    final_cuan = df_stats["Pension Balance After Fees (€) (CUAN)"].iloc[-1]
    final_auto = df_stats["Pension Balance After Fees (€) (Auto-Enrolment)"].iloc[-1]
    fees_cuan = df_stats["Total Fees Earned (€) (CUAN)"].iloc[-1]
    fees_auto = df_stats["Total Fees Earned (€) (Auto-Enrolment)"].iloc[-1]
    avg_contribution_cuan = df_stats["Annual Contribution (€) (CUAN)"].mean()
    avg_contribution_auto = df_stats["Annual Contribution (€) (Auto-Enrolment)"].mean()

    # Compute the yearly balance difference and determine the maximum difference and corresponding year
    df_stats["Balance Difference (CUAN - Auto)"] = (
        df_stats["Pension Balance After Fees (€) (CUAN)"] - df_stats["Pension Balance After Fees (€) (Auto-Enrolment)"]
    )
    max_diff = df_stats["Balance Difference (CUAN - Auto)"].max()
    year_max_diff = df_stats.loc[df_stats["Balance Difference (CUAN - Auto)"].idxmax(), "Year"]

    # Relative percentage difference between final balances
    relative_diff = (final_cuan - final_auto) / final_auto * 100

    # -------------------------------------------------------------------
    # DEFINE THE EXTRA VARIABLES BEFORE PRINTING THEM
    # -------------------------------------------------------------------

    # 1) Compute CAGR (Compound Annual Growth Rate) for CUAN & Auto-Enrolment
    #    - initial balance is from row 0, "After Fees" columns
    initial_cuan = df_stats["Pension Balance After Fees (€) (CUAN)"].iloc[0]
    initial_auto = df_stats["Pension Balance After Fees (€) (Auto-Enrolment)"].iloc[0]

    #    - number of years is the last "Year" value in the DataFrame
    sim_years = df_stats["Year"].iloc[-1]  # e.g., if final row is Year 45, sim_years = 45

    CAGR_cuan = (final_cuan / initial_cuan) ** (1 / sim_years) - 1 if initial_cuan > 0 else 0
    CAGR_auto = (final_auto / initial_auto) ** (1 / sim_years) - 1 if initial_auto > 0 else 0

    # 2) Total Contributions
    total_contrib_cuan = df_stats["Annual Contribution (€) (CUAN)"].sum()
    total_contrib_auto = df_stats["Annual Contribution (€) (Auto-Enrolment)"].sum()

    # 3) Contribution Multiplier = Final Balance / Total Contributions
    multiplier_cuan = final_cuan / total_contrib_cuan if total_contrib_cuan > 0 else 0
    multiplier_auto = final_auto / total_contrib_auto if total_contrib_auto > 0 else 0

    # 4) Fees as Percentage of Gross Accumulation
    #    - Gross = Final Balance + Total Fees Earned
    gross_cuan = final_cuan + fees_cuan
    gross_auto = final_auto + fees_auto
    fees_pct_cuan = (fees_cuan / gross_cuan) * 100 if gross_cuan > 0 else 0
    fees_pct_auto = (fees_auto / gross_auto) * 100 if gross_auto > 0 else 0

    # -------------------------------------------------------------------
    # NOW PRINT ALL THE STATS
    # -------------------------------------------------------------------
    print("===== Summary Statistics =====")
    print(f"Final Pension Balance (CUAN):                {final_cuan:,.2f} €")
    print(f"Final Pension Balance (Auto-Enrolment):      {final_auto:,.2f} €")
    print()
    print(f"Total Fees Earned (CUAN):                    {fees_cuan:,.2f} €")
    print(f"Total Fees Earned (Auto-Enrolment):          {fees_auto:,.2f} €")
    print()
    print(f"Average Annual Contribution (CUAN):          {avg_contribution_cuan:,.2f} €")
    print(f"Average Annual Contribution (Auto-Enrolment):{avg_contribution_auto:,.2f} €")
    print()
    print(f"Year with Maximum Balance Difference:        {year_max_diff}")
    print(f"Maximum Balance Difference (CUAN - Auto):    {max_diff:,.2f} €")
    print(f"Relative Percentage Difference:              {relative_diff:.2f}%")
    print()
    print(f"CAGR (CUAN):                                {CAGR_cuan*100:.2f}% per year")
    print(f"CAGR (Auto-Enrolment):                      {CAGR_auto*100:.2f}% per year")
    print()
    print(f"Total Contributions (CUAN):                 {total_contrib_cuan:,.2f} €")
    print(f"Total Contributions (Auto-Enrolment):       {total_contrib_auto:,.2f} €")
    print()
    print(f"Contribution Multiplier (CUAN):             {multiplier_cuan:.2f}x")
    print(f"Contribution Multiplier (Auto-Enrolment):   {multiplier_auto:.2f}x")
    print()
    print(f"Fees as Percentage of Gross Accumulation (CUAN):   {fees_pct_cuan:.2f}%")
    print(f"Fees as Percentage of Gross Accumulation (Auto-Enrolment): {fees_pct_auto:.2f}%")

    # Optionally, display the final row again for visual confirmation
    print("\nFinal row from df:")
    print(df.tail(1).to_string(index=False))